In [2]:
# Multilayer Perceptron
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

batch_size = 200
learning_rate = 0.01
epochs = 5

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                  transform=transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,), (0.3081,))
                  ])),
    batch_size = batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size = batch_size, shuffle=True)

class MLP(nn.Module):
    
    def __init__(self):
        super(MLP, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(),
            nn.Linear(200, 200),
            nn.LeakyReLU(),
            nn.Linear(200, 10),
            nn.LeakyReLU()
        )

    def forward(self, x):
        x = self.model(x)
        return x
    
net = MLP()
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        
        y_hat = net(data)
        loss = criterion(y_hat, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print("Train epoch : {} [{}/{} ({:.0f}%)] \t Loss:{:.6f}".format(epoch, batch_idx * len(data),
                                                                            len(train_loader.dataset),
                                                                            100. * batch_idx / len(train_loader), loss.item()))
    test_loss = 0
    correct = 0
        
    for data, target in test_loader:
        data = data.view(-1, 28*28)
        y_hat = net(data)
        test_loss += criterion(y_hat, target).item()
        pred = y_hat.argmax(dim=1)
        correct += pred.eq(target).float().sum().item()
        
    test_loss /= len(test_loader.dataset)
    print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train epoch : 0 [0/60000 (0%)] 	 Loss:2.314842
Train epoch : 0 [20000/60000 (33%)] 	 Loss:2.128735
Train epoch : 0 [40000/60000 (67%)] 	 Loss:1.671124

Test set: Average loss: 0.0067, Accuracy: 5948.0/10000 (59%)

Train epoch : 1 [0/60000 (0%)] 	 Loss:1.393217
Train epoch : 1 [20000/60000 (33%)] 	 Loss:1.242660
Train epoch : 1 [40000/60000 (67%)] 	 Loss:0.975820

Test set: Average loss: 0.0044, Accuracy: 7178.0/10000 (72%)

Train epoch : 2 [0/60000 (0%)] 	 Loss:0.959363
Train epoch : 2 [20000/60000 (33%)] 	 Loss:0.741190
Train epoch : 2 [40000/60000 (67%)] 	 Loss:0.726640

Test set: Average loss: 0.0039, Accuracy: 7543.0/10000 (75%)

Train epoch : 3 [0/60000 (0%)] 	 Loss:0.908970
Train epoch : 3 [20000/60000 (33%)] 	 Loss:0.751757
Train epoch : 3 [40000/60000 (67%)] 	 Loss:0.740725

Test set: Average loss: 0.0019, Accuracy: 8987.0/10000 (90%)

Train epoch : 4 [0/60000 (0%)] 	 Loss:0.452921
Train epoch : 4 [20000/60000 (33%)] 	 Loss:0.300658
Train epoch : 4 [40000/60000 (67%)] 	 Loss:0.